<a href="https://colab.research.google.com/github/wqiu96/MA574_Portfolio-Valuation-and-Risk-Management/blob/master/src/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import os

In [2]:
!git clone https://github.com/wqiu96/MA574_Portfolio-Valuation-and-Risk-Management.git

Cloning into 'MA574_Portfolio-Valuation-and-Risk-Management'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 581 (delta 7), reused 0 (delta 0), pack-reused 560
Receiving objects: 100% (581/581), 3.34 MiB | 7.73 MiB/s, done.
Resolving deltas: 100% (234/234), done.


In [3]:
cd MA574_Portfolio-Valuation-and-Risk-Management/src/project4/

/content/MA574_Portfolio-Valuation-and-Risk-Management/src/project4


In [0]:
#Load data
os.chdir('/content/MA574_Portfolio-Valuation-and-Risk-Management/src/project4/')
file_chdir = os.getcwd()
filecsv_list = []
for root,dirs,files in os.walk(file_chdir):
  for file in files:
    if os.path.splitext(file)[1] == '.xlsx':
      filecsv_list.append(file)
R = pd.DataFrame()

for xls in filecsv_list:
  if os.path.splitext(xls)[0] == 'GDP':
    R[xls.strip('.xlsx')] = pd.read_excel(xls)['Last Price'][0:-1].values
  else:
    R[xls.strip('.xlsx')] = pd.read_excel(xls)['Last Price'][1:].values

In [14]:
filecsv_list

['GDP.xlsx',
 'GOVT PURCHASE and INVESTMENT.xlsx',
 'US URBAN CPI.xlsx',
 'Core Price Consumption.xlsx',
 'US FEDERAL FUNDS TARGET RATE.xlsx',
 'Private Domestic Investment.xlsx',
 'US Trade Balance.xlsx']

In [18]:
R

,GDP,GOVT PURCHASE and INVESTMENT,US URBAN CPI,Core Price Consumption,US FEDERAL FUNDS TARGET RATE,Private Domestic Investment,US Trade Balance
0,2.9,2.1,0.1,1.6,2.25,0.703336,1.2
1,4.8,2.6,0.2,2.1,2.00,2.016887,0.3
2,7.1,1.9,0.1,2.3,1.75,2.185899,-0.9
3,5.0,2.4,0.2,2.2,1.50,2.423938,2.0
4,6.4,-0.1,0.5,1.5,1.25,0.869080,1.0
5,5.4,1.4,0.1,1.3,1.25,1.261745,0.4
6,3.5,-0.2,-0.1,1.7,1.00,2.298477,0.5
7,4.2,1.1,0.3,1.4,0.75,1.007472,1.1
8,4.0,1.7,0.2,2.0,0.50,1.134094,-0.9
9,3.7,-0.7,0.3,2.0,0.50,1.017987,2.0


In [20]:
Y = R['GDP'].values
Par = R.copy()
Par['GDP'] = np.ones(Par.shape[0])
Par.columns = ['Beta_0', 'GOV_INV', 'CPI','CONSUMPTION','FED_RATE','PRIVATE_INV','TRADE']
data = Par.join(R['GDP'])
X = Par.values
B = np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)),X.T),Y)
B

array([ 3.35956651, -0.0641456 ,  1.93828198, -0.44860675,  0.8712587 ,
        0.39080227,  0.12532735])

In [21]:
# load packages
import statsmodels.api as sm
from statsmodels.formula.api import ols
# Ordinary Least Squares (OLS) model
model = ols('GDP ~ GOV_INV + CPI + CONSUMPTION + FED_RATE + PRIVATE_INV + TRADE', data=data).fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    GDP   R-squared:                       0.282
Model:                            OLS   Adj. R-squared:                  0.147
Method:                 Least Squares   F-statistic:                     2.094
Date:                Sun, 10 Nov 2019   Prob (F-statistic):             0.0816
Time:                        20:01:20   Log-Likelihood:                -75.951
No. Observations:                  39   AIC:                             165.9
Df Residuals:                      32   BIC:                             177.5
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       3.3596      1.026      3.273      0.003       1.269       5.450
GOV_INV        -0.0641      0.147     -0.435      0.666      -0.364       0.236
CPI             1.9383      1.540      1.258      0.217      -1.199       5.076
CONSUMPTION    -0.4486      0.681     -0.659      0.515      -1.835       0.938
FED_RATE        0.8713      0.644      1.353      0.185      -0.440       2.183
PRIVATE_INV     0.3908      0.181      2.160      0.038       0.022       0.759
TRADE           0.1253      0.161      0.778      0.442      -0.203       0.453
==============================================================================
Omnibus:                        0.265   Durbin-Watson:                   2.392
Prob(Omnibus):                  0.876   Jarque-Bera (JB):                0.237
Skew:                          -0.168   Prob(JB):                        0.888
Kurtosis:                       2.818   Cond. No.                         16.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

pd.Series([variance_inflation_factor(Par.values, i) 
               for i in range(Par.shape[1])], 
              index=Par.columns)

Beta_0         11.712072
GOV_INV         1.630122
CPI             1.308576
CONSUMPTION     1.353627
FED_RATE        1.297006
PRIVATE_INV     1.778324
TRADE           1.231445
dtype: float64